In [ ]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Function to calculate distance between two lat-lng points
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    # Distance in kilometers
    distance = R * c
    return distance
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'

In [ ]:
df = pd.read_csv('merged_rides__with_elevation.csv')


In [ ]:
df['Start_Altitude'] = df[['Start_Altitude', 'Start_Altutude', 'State_Altitude']].bfill(axis=1).iloc[:, 0]
df.drop(columns=['State_Altitude', 'Start_Altutude'], inplace=True)

     

df['End_Altitude'] = df[['End_Altitude', 'end_altitude', 'end_Altitude']].bfill(axis=1).iloc[:, 0]
     

df.drop(columns=['end_Altitude', 'end_altitude'], inplace=True)
     

df['Start_Altitude'].fillna(method='ffill', inplace=True)
df['End_Altitude'].fillna(df['End_Altitude'].mean(), inplace=True)

In [ ]:
df.head()

df['Start_Altitude'] = df[['Start_Altitude', 'Start_Altutude', 'State_Altitude']].bfill(axis=1).iloc[:, 0]
     

df.drop(columns=['State_Altitude', 'Start_Altutude'], inplace=True)

     

df['End_Altitude'] = df[['End_Altitude', 'end_altitude', 'end_Altitude']].bfill(axis=1).iloc[:, 0]
     

df.drop(columns=['end_Altitude', 'end_altitude'], inplace=True)
     

df['Start_Altitude'].fillna(method='ffill', inplace=True)
df['End_Altitude'].fillna(df['End_Altitude'].mean(), inplace=True)
     

from sklearn.impute import SimpleImputer
df['Elevation_Change'] = df['End_Altitude'] - df['Start_Altitude']
     

imputer = SimpleImputer(strategy='mean')
df['Elevation_Change'] = imputer.fit_transform(df[['Elevation_Change']])
     

df = df.drop(columns=['Start_Altitude', 'End_Altitude'])
     

df['Distance'] = df.apply(lambda x: haversine_distance(x['start_lat'], x['start_lng'], x['end_lat'], x['end_lng']), axis=1)
     

df = df.drop(columns=['start_lat', 'start_lng', 'end_lat', 'end_lng'])
     

df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [ ]:
df['season'] = df['started_at'].dt.month.map(get_season)
     

def get_day_of_week(timestamp):
    return timestamp.strftime("%A")
     

# Apply the function to each row in the DataFrame
df['day_of_week'] = df['started_at'].apply(get_day_of_week)
     

df['trip_duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
     

weatherData = '/content/drive/My Drive/3667056.csv'
     

tempData = pd.read_csv(weatherData)
     

tempData['started_at'] = pd.to_datetime(tempData['DATE'])
tempData['started_at'] = tempData['started_at'].dt.date
     

df['started_at'] = df['started_at'].dt.date
     

df = pd.merge(df, tempData[['started_at', 'TMAX', 'TMIN']], on='started_at', how='left')
     

# ride_types = dfs['rideable_type'].unique()

In [ ]:
df.fillna(df.mean(numeric_only=True), inplace=True)  # Only fill numeric columns with their means
df.fillna('Unknown', inplace=True)
     

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
     

target_variable = 'rideable_type'
y = df[target_variable]
X = df.drop(target_variable, axis=1)
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object':  # This identifies categorical columns
        num_unique_values = len(X[column].unique())
        if num_unique_values < 10:  # Limit set for one-hot encoding
            # Apply one-hot encoding to columns with fewer unique values
            dummies = pd.get_dummies(X[column], prefix=column)
            X = pd.concat([X, dummies], axis=1)
        else:
            # Apply label encoding to columns with many unique values to save memory
            label_encoders[column] = LabelEncoder()
            X[column] = label_encoders[column].fit_transform(X[column].astype(str))
        X.drop(column, axis=1, inplace=True)  # Drop original column after encoding
     

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
     

y_train_encoded = (y_train == 'electric').astype(int)  # Encode 'electric' as 1, 'classic' as 0
y_test_encoded = (y_test == 'electric').astype(int)
     

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
     

model = LinearRegression()

     

X_train.drop(columns=['ended_at'], inplace=True)

In [ ]:
print(X_train.dtypes)
print(y_train_encoded)

Elevation_Change         float64
Distance                 float64
trip_duration            float64
TMAX                       int64
TMIN                       int64
member_casual_Unknown       bool
member_casual_casual        bool
member_casual_member        bool
season_Fall                 bool
season_Spring               bool
season_Summer               bool
season_Winter               bool
day_of_week_Friday          bool
day_of_week_Monday          bool
day_of_week_Saturday        bool
day_of_week_Sunday          bool
day_of_week_Thursday        bool
day_of_week_Tuesday         bool
day_of_week_Wednesday       bool
dtype: object
2220921     1
6189410     0
587405      1
10376738    0
3510934     0
           ..
2234489     1
4304572     1
10081351    1
6550634     1
6423388     1
Name: rideable_type, Length: 7971288, dtype: int64


In [ ]:
X_test.drop(columns=['ended_at'], inplace=True)
     

# Fit the model on the training data with encoded labels
model.fit(X_train, y_train_encoded)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
     

gb_regressor = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)  # Hyperparameters can be adjusted
gb_regressor.fit(X_train, y_train_encoded)

GradientBoostingRegressor(random_state=42)

In [ ]:
y_pred = gb_regressor.predict(X_test)
     

feature_importances = gb_regressor.feature_importances_

     

feature_names = X_train.columns
     

importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances}).sort_values(by='Importance', ascending=False)
print(importances_df)

Feature  Importance
1                Distance    0.498346
2           trip_duration    0.416795
0        Elevation_Change    0.050170
6    member_casual_casual    0.019550
7    member_casual_member    0.010294
4                    TMIN    0.002744
3                    TMAX    0.000775
8             season_Fall    0.000307
10          season_Summer    0.000297
9           season_Spring    0.000266
12     day_of_week_Friday    0.000125
15     day_of_week_Sunday    0.000122
11          season_Winter    0.000104
14   day_of_week_Saturday    0.000092
13     day_of_week_Monday    0.000013
5   member_casual_Unknown    0.000000
16   day_of_week_Thursday    0.000000
17    day_of_week_Tuesday    0.000000
18  day_of_week_Wednesday    0.000000

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
     

# Assuming gb_regressor is your trained gradient boosting regression model
predictions = gb_regressor.predict(X_test)  # X_test is your test data

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_encoded, y_pred)
mse = mean_squared_error(y_test_encoded, y_pred)
rmse = mean_squared_error(y_test_encoded, y_pred, squared=False)  # RMSE
r2 = r2_score(y_test_encoded, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)

Mean Absolute Error: 0.41125352854674235
Mean Squared Error: 0.19575479960088926
Root Mean Squared Error: 0.44244186013632264
R-squared: 0.21649823870180052